MATA Data project code.

This code still needs to be cleaned up, but it is all there. 



In [ ]:
import numpy as np

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [ ]:
engine = create_engine("sqlite:////Users/nhuvelle/Documents/mta_data.db")

In [ ]:
insp = inspect(engine)
print(insp.get_table_names())

In [ ]:
df = pd.read_sql('SELECT * FROM mta_data WHERE (STATION == "WORLD TRADE CTR" or STATION == "TIMES SQ-42 ST" or STATION == "WHITEHALL S-FRY" or STATION == "81 ST-MUSEUM") AND DATE LIKE "%2021" ORDER BY STATION, DATE', engine)
df

In [ ]:
import datetime
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, format="%m/%d/%Y %H:%M:%S")
df

In [ ]:
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)
df

In [ ]:
df = df.drop(["EXITS", "DESC", "LINENAME", "DIVISION"], axis=1, errors="ignore")
df.head()

In [ ]:
df[["PREV_DATE_TIME", "PREV_ENTRIES"]] = (df.groupby(["STATION"])[["DATE_TIME", "ENTRIES"]].shift(1))
df.head()

In [ ]:
#Drop the rows for the earliest date in the df
df.dropna(subset=["PREV_DATE_TIME"], axis=0, inplace=True)
df.head()

In [ ]:
turnstiles_daily = (df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False).ENTRIES.first())
turnstiles_daily.head()

In [ ]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE", "ENTRIES"]]
                                                       .shift(1))
turnstiles_daily.head()

In [ ]:
#Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
(turnstiles_daily["ENTRIES"] - turnstiles_daily["PREV_ENTRIES"]).describe()

In [ ]:
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].shape

In [ ]:
turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

In [ ]:
# Let's see how many stations have this problem
(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
     .groupby(["C/A", "UNIT", "SCP", "STATION"])
     .size())

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    
    if counter < 0:
        counter = -counter  # adjust for "reverse" counter
        
    if counter > max_counter:
        # Maybe counter was reset, so it may make sense to take the minimum
        print(f'entries: {row["ENTRIES"]} <-- {row["PREV_ENTRIES"]}')
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
        
    if counter > max_counter:
        # If we still get a counter that is too big, set to zero
        return 0
    
    return counter

turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
turnstiles_daily.describe()

In [ ]:
turnstiles_daily.DAILY_ENTRIES.describe()

In [ ]:
station_daily = turnstiles_daily.groupby(["C/A", "UNIT", "STATION", "DATE"])[['DAILY_ENTRIES']].sum().reset_index()
station_daily.head()

In [ ]:
station_daily = turnstiles_daily.groupby(["STATION", "DATE"])[['DAILY_ENTRIES']].sum().reset_index()
station_daily.tail()

In [ ]:
station_totals = station_daily.groupby('STATION')['DAILY_ENTRIES'].sum() \
    .reset_index() \
    .sort_values('DAILY_ENTRIES', ascending=False)

station_totals.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']  # or retina
%matplotlib inline

sns.set(context='notebook', 
    style='whitegrid', 
    font_scale=1.1)

In [ ]:
station_daily_MUSEUM_av = station_daily[station_daily['STATION'] == '81 ST-MUSEUM'].copy()
station_daily_MUSEUM_av.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_MUSEUM_av['DATE'], station_daily_MUSEUM_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for 81 ST-MUSEUM')

In [ ]:
station_daily_MUSEUM_av['DAILY_ENTRIES'].max()

In [ ]:
station_daily_MUSEUM_av[station_daily_MUSEUM_av['DAILY_ENTRIES'] == station_daily_MUSEUM_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_MUSEUM_av.describe()
station_daily_MUSEUM_av.shape

In [ ]:
station_daily_MUSEUM_av.drop(111, axis=0, inplace=True)
station_daily_MUSEUM_av.describe()

In [ ]:
#station_daily_MUSEUM_av.head(31)
station_daily_MUSEUM_av.iloc[0:30]['DAILY_ENTRIES'].mean()

In [ ]:
station_daily_MUSEUM_av.iloc[30:58]['DAILY_ENTRIES'].mean()

In [ ]:
station_daily_MUSEUM_av.iloc[58:89]['DAILY_ENTRIES'].mean()

In [ ]:
station_daily_MUSEUM_av.iloc[89:118]['DAILY_ENTRIES'].mean()

In [ ]:
station_daily_MUSEUM_av.iloc[118:149]['DAILY_ENTRIES'].mean()

In [ ]:
station_daily_MUSEUM_av.iloc[149:]['DAILY_ENTRIES'].mean()

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','June']
museum_months = [2272,2309,2713,3199,3758,4221]
plt.bar(months, museum_months)
plt.ylabel('Average Daily Entries',family = 'fantasy',color='red')
plt.title('Daily Entries for 81 ST-MUSEUM', fontweight = 'bold');


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_MUSEUM_av['DATE'], station_daily_MUSEUM_av['DAILY_ENTRIES'])
plt.ylabel('# of Daily Entries',family = 'fantasy', color='red')
#plt.xlabel('Date')
x_ticks = np.linspace(0, 150, 6)
months = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(x_ticks, months, fontsize = '20', family = 'fantasy',color='red')
plt.title('Daily Entries for 81 ST-MUSEUM', fontweight = 'bold')
#plt.grid(b=None)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_MUSEUM_av['DATE'], station_daily_MUSEUM_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
x_ticks = np.linspace(0, 150, 6)
months = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(x_ticks, months, fontsize = '20', family = 'fantasy',color='red')
#plt.xticks(rotation=45)
plt.title('Daily Entries for 81 ST-MUSEUM')

In [ ]:
station_daily_42_ST_av = station_daily[station_daily['STATION'] == 'TIMES SQ-42 ST'].copy()
station_daily_42_ST_av.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_42_ST_av['DATE'], station_daily_42_ST_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for TIMES SQ 42-ST Station')

In [ ]:
station_daily_42_ST_av[station_daily_42_ST_av['DAILY_ENTRIES'] == station_daily_42_ST_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_42_ST_av.drop(266, axis=0, inplace=True)
station_daily_42_ST_av.describe()

In [ ]:
station_daily_42_ST_av[station_daily_42_ST_av['DAILY_ENTRIES'] == station_daily_42_ST_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_42_ST_av.describe()

In [ ]:
station_daily_42_ST_av[station_daily_42_ST_av['DAILY_ENTRIES'] == station_daily_42_ST_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_42_ST_av.drop(196, axis=0, inplace=True)
station_daily_42_ST_av.describe()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_42_ST_av['DATE'], station_daily_42_ST_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for TIMES SQ 42-ST Station')

In [ ]:
station_daily_42_ST_av[station_daily_42_ST_av['DAILY_ENTRIES'] == station_daily_42_ST_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_42_ST_av.drop(225, axis=0, inplace=True)
station_daily_42_ST_av.describe()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_42_ST_av['DATE'], station_daily_42_ST_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for TIMES SQ 42-ST Station')

In [ ]:
station_daily_42_ST_av[station_daily_42_ST_av['DAILY_ENTRIES'] == station_daily_42_ST_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_42_ST_av.drop(265, axis=0, inplace=True)
station_daily_42_ST_av.describe()

In [ ]:
station_daily_42_ST_av.iloc[0:30].mean()

In [ ]:
station_daily_42_ST_av.iloc[30:56].mean()

In [ ]:
station_daily_42_ST_av.iloc[56:86].mean()

In [ ]:
station_daily_42_ST_av.iloc[86:114].mean()

In [ ]:
station_daily_42_ST_av.iloc[114:145].mean()

In [ ]:
station_daily_42_ST_av.iloc[145:].mean()

In [ ]:
plt.figure(figsize=(9, 4))
plt.plot(station_daily_42_ST_av['DATE'], station_daily_42_ST_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries',family = 'fantasy',color='red')
plt.xlabel('Date')
x_ticks = np.linspace(0, 150, 6)
months = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(x_ticks, months, fontsize = '20', family = 'fantasy',color='red')
#plt.xticks(rotation=45)
plt.title('Daily Entries for TIMES SQ 42-ST Station', fontweight = 'bold')

In [ ]:
months2 = ['Jan','Feb','Mar','Apr','May','June']
museum_months2 = [13363,12658,15772,17473,19443,22156]
plt.ylabel('Average Daily Entries',family = 'fantasy',color='red')
plt.bar(months2, museum_months2)
plt.title('Daily Entries for TIMES SQ 42-ST Station', fontweight = 'bold')

In [ ]:
station_daily_WHITEHALL_av.iloc[0:30].mean()

In [ ]:
station_daily_WHITEHALL_av.iloc[30:58].mean()

In [ ]:
station_daily_WHITEHALL_av.iloc[58:89].mean()

In [ ]:
station_daily_WHITEHALL_av.iloc[89:119].mean()

In [ ]:
station_daily_WHITEHALL_av.iloc[119:150].mean()

In [ ]:
station_daily_WHITEHALL_av.iloc[150:].mean()

In [ ]:
months3 = ['Jan','Feb','Mar','Apr','May','June']
museum_months3 = [2531,2529,3017,3354,3788,4294]
plt.bar(months3, museum_months3)
plt.ylabel('Average Daily Entries',family = 'fantasy',color='red')
plt.title('Daily Entries for WHITEHALL S-FRY', fontweight = 'bold');

In [ ]:
station_daily_WHITEHALL_av = station_daily[station_daily['STATION'] == 'WHITEHALL S-FRY'].copy()
station_daily_WHITEHALL_av.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_WHITEHALL_av['DATE'], station_daily_WHITEHALL_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries',family = 'fantasy',color='red')
x_ticks = np.linspace(0, 150, 6)
months = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(x_ticks, months, fontsize = '20', family = 'fantasy',color='red')
#plt.xticks(rotation=45)
plt.title('Daily Entries for WHITEHALL S-FRY Station', fontweight='bold')

In [ ]:
station_daily_WORLD_TRADE_av = station_daily[station_daily['STATION'] == 'WORLD TRADE CTR'].copy()
station_daily_WORLD_TRADE_av.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_WORLD_TRADE_av['DATE'], station_daily_WORLD_TRADE_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for WORLD TRADE CTR Station')

In [ ]:
station_daily_WORLD_TRADE_av[station_daily_WORLD_TRADE_av['DAILY_ENTRIES'] == station_daily_WORLD_TRADE_av['DAILY_ENTRIES'].max()]

In [ ]:
station_daily_WORLD_TRADE_av.drop(529, axis=0, inplace=True)
station_daily_WORLD_TRADE_av.describe()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_WORLD_TRADE_av['DATE'], station_daily_WORLD_TRADE_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries',family = 'fantasy',color='red')
x_ticks = np.linspace(0, 150, 6)
months = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(x_ticks, months, fontsize = '20', family = 'fantasy',color='red')
plt.title('Daily Entries for WORLD TRADE CTR Station', fontweight='bold');

In [ ]:
station_daily_WORLD_TRADE_av.iloc[0:30].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[30:57].mean()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(station_daily_WORLD_TRADE_av['DATE'], station_daily_WORLD_TRADE_av['DAILY_ENTRIES'])
plt.ylabel('# of Entries')
plt.xlabel('Date')
#plt.xticks(rotation=45)
plt.title('Daily Entries for WORLD TRADE CTR Station')

In [ ]:
station_daily_WORLD_TRADE_av.iloc[0:30].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[30:57].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[57:88].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[88:118].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[118:149].mean()

In [ ]:
station_daily_WORLD_TRADE_av.iloc[149:].mean()

In [ ]:
months4 = ['Jan','Feb','Mar','Apr','May','June']
museum_months4 = [1608,1492,1873,2317,2810,3402]
plt.bar(months4, museum_months4)
plt.ylabel('Average Daily Entries',family = 'fantasy',color='red')
plt.title('Daily Entries for WORLD TRADE CTR', fontweight = 'bold');